# GLE dynamics with lammps

In this notebook we use LAMMPS to run a simulation of particles subject to the GLE. Lammps already has two "fixes" which implement the generalized Langevin equation **fix gle** and **fix gld**. 

**fix gld** is based off formalism presented in a paper by Baczewski and Bond (A.D. Baczewski and S.D. Bond, J. Chem. Phys. 139, 044107 (2013).). This formalism is assumes the memory kernel is a sum of exponential functions (prony series). 

**fix gle** is based off formalism presented in a paper by Ceriotti et al. (Ceriotti, Bussi and Parrinello, J Chem Theory Comput 6, 1170-80 (2010)). This formalism is more general than that of Baczewski and Bond, and indeed can be shown to be equivalent to Baczewski and Bond for certain choices of the $\mathbf{A}$ parameter matrix. 

Both fixes assume the memory kernel is **isotropic**, that is the memory kernel acting on each cartesian component of each particle is equivalent. They also do not allow for memory to be associated to rotational degrees of freedom. These capabilities require a modification of the LAMMPS source code. 

In this script we will use **fix gle** to implement a simulation of a cluster of ions in Debye-Huckel implicit solvent subject to a colored noise bath.

In [1]:
from lammps import lammps
from mpi4py import MPI

In [2]:
##### Script parameters
time_step = 1.0
eq_steps  = 50000
run_steps = 250000

##### Run simulation
lmp = lammps()

block = """
# LAMMPS script file to simulate implicit solvent ion box

units       real
dimension   3
boundary    p p p

atom_style  full
pair_style	lj/cut/coul/debye 0.3378 12 12

read_data "ion_implicit.dat"

group cation type 1
group anion type 2
group ion type 1 2

pair_modify shift yes mix arithmetic
# i j epsilon sigma
pair_coeff 1 1 0.33673 1.4094	#Li+  
pair_coeff 2 2 0.01279 4.8305	#Cl-  

dielectric   70.7

# Set up integration
timestep %0.1f  # fs
run_style verlet
neighbor 2.0 bin
neigh_modify delay 0 every 1 check yes

##################
# Equilibration
##################

fix cation_gle cation gle 10 300 300 31415 K_cation.A
fix anion_gle anion gle 10 300 300 11415 K_anion.A

thermo_style custom step temp press vol etotal
thermo 1000

# run equilibriation
print 'Running equilibriation'
run %d

##################
# Production
##################

# Dumps
dump the_big_dump ion custom 5 implicit_run.lammpstrj id type x y z vx vy vz fx fy fz

# Run production
print 'Running production'
run %d
"""%(time_step,eq_steps,run_steps)

lmp.commands_string(block)

LAMMPS (23 Jun 2022 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0 0 0) to (25 25 25)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  18 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.000 seconds
  read_data CPU = 0.015 seconds
9 atoms in group cation
9 atoms in group anion
18 atoms in group ion
Running equilibriation
Generated 1 of 1 mixed pair_coeff terms from arithmetic mixing rule
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 14
  ghost atom cutoff = 14
  binsize = 7, bins = 4 4 4
  1 n

## Note 
Note that because lammps does not add the forces from the GLE fix to fx/fy/fz, trying to extract a memory kernel from the output data will lead to incorrect results.